In [1]:
import pandas as pd
import time
import requests
import json
from tqdm import tqdm

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument("--start-maximized")
options.add_argument("headless")
s = Service(ChromeDriverManager().install())
chrome = webdriver.Chrome(options = options,service = s)



====== WebDriver manager ======
Current google-chrome version is 106.0.5249
Get LATEST chromedriver version for 106.0.5249 google-chrome
Driver [/home/cosbi/.wdm/drivers/chromedriver/linux64/106.0.5249.61/chromedriver] found in cache


In [2]:
chrome.get("https://tw.stock.yahoo.com/h/kimosel.php?tse=1&cat=%A5b%BE%C9%C5%E9&form=menu&form_id=stock_id&form_name=stock_name&domain=0")

In [3]:
ticker = {}
td_text = chrome.find_elements(by = By.TAG_NAME, value = "td")[3].text
tiker_row = td_text.split("\n")
tiker_row[0] = tiker_row[0].replace("上市 ", "")
tiker_row[len(tiker_row) - 1] = tiker_row[len(tiker_row) - 1][:-18]

In [4]:
for row in tiker_row:
    cols = row.split(" ")

    for col in cols: ticker[col] = []

In [6]:
import json
list(ticker.keys())

['水泥',
 '食品',
 '塑膠',
 '紡織',
 '電機',
 '電器電纜',
 '化學',
 '生技',
 '玻璃',
 '造紙',
 '鋼鐵',
 '橡膠',
 '上櫃',
 '汽車',
 '半導體',
 '電腦週邊',
 '光電',
 '通訊網路',
 '電子零組件',
 '電子通路',
 '資訊服務',
 '其他電子',
 '營建',
 '航運',
 '觀光',
 '金融業',
 '貿易百貨',
 '油電燃氣',
 '存託憑證',
 'ETF',
 '受益證券',
 'ETN',
 '創新板',
 '其他',
 '市認購',
 '市認售',
 '指數類',
 '市牛證',
 '市熊證',
 '戰略新板']

In [5]:
for key in tqdm(ticker.keys()):
    chrome.get("https://tw.stock.yahoo.com/h/kimosel.php?tse=1&cat=" + key + "&form=menu&form_id=stock_id&form_name=stock_name&domain=0")
    tables_tag = chrome.find_elements(by = By.TAG_NAME, value = "table")[6]
    a_tags = tables_tag.find_elements(by = By.TAG_NAME, value = "a")

    for a_tag in a_tags: ticker[key].append(a_tag.text)

    time.sleep(0.5)


100%|██████████| 40/40 [15:57<00:00, 23.93s/it] 


In [8]:
a = []
b = []

for key in ticker.keys():
    a.extend(ticker[key])
    b.extend([key for i in range(len(ticker[key]))])

In [9]:
df = pd.DataFrame({ "ticker" : a, "class" : b })
df

,ticker,class
0,1101 台泥,水泥
1,1101B 台泥乙特,水泥
2,1102 亞泥,水泥
3,1103 嘉泥,水泥
4,1104 環泥,水泥
...,...,...
26002,6912 益鈞環科*-新,戰略新板
26003,6913 鴻呈-新,戰略新板
26004,6922 宸曜-新,戰略新板
26005,6925 意藍-新,戰略新板


In [10]:
df.to_csv("ticker_list.csv", index = False)